In [2]:
import torch
torch.set_printoptions(precision=15)
from torch import nn
import numpy as np
import sys
sys.path.append('../../l-system-captioning/')
sys.path.append('../')

from lsystem_modules import SlimLstm

vocab = torch.load('../../l-system-captioning/data/Simplest_48267_256_max_len_+F_vocab.pth')

model = SlimLstm.load_from_checkpoint('../../l-system-captioning/checkpoints/slim_lstm_rand/last.ckpt')

Lightning automatically upgraded your loaded checkpoint from v1.8.5.post0 to v1.9.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file c:\Users\magnusson\git\l-system-captioning\checkpoints\slim_lstm_rand\last.ckpt`


In [3]:
test_cell = nn.LSTMCell(256,256)

In [4]:
test_cell.weight_ih = model.decoder.lstm.weight_ih_l0
test_cell.weight_hh = model.decoder.lstm.weight_hh_l0
test_cell.bias_ih   = model.decoder.lstm.bias_ih_l0
test_cell.bias_hh   = model.decoder.lstm.bias_hh_l0

In [5]:
x = torch.rand((3,256)).double()
x.requires_grad = True

In [6]:
from lrp.functional.lstm import LstmAll

In [7]:
test_cell.eps = torch.tensor(0.001)
test_cell.bias_factor = torch.tensor(0.0)

a = torch.zeros(256).double()
outs = torch.zeros_like(x)
out_a, h_a, c_a = LstmAll.apply(x[0, None], a[None], a[None], test_cell)
outs[0] = out_a
out_a, h_a, c_a = LstmAll.apply(x[1, None], h_a, c_a, test_cell)
outs[1] = out_a
out_a, h_a, c_a = LstmAll.apply(x[2, None], h_a, c_a, test_cell)
outs[2] = out_a

In [8]:
# h_a.sum().backward()
(out_a.sum() / 256).backward()

In [9]:
from lrp.functional._lstm import LSTM_bidi

lstm = LSTM_bidi(model.decoder.lstm)

In [10]:
Rx = lstm.lrp(x.detach().numpy())

In [11]:
torch.testing.assert_close(out_a, torch.tensor(lstm.s)[None])

In [12]:
R_x = x.grad
torch.testing.assert_close(R_x, torch.tensor(Rx))

In [13]:
from lrp.lstm import LSTM

l = LSTM.from_torch(model.decoder.lstm)


In [14]:
i = x.detach()
i.requires_grad = True
out, (h, c) = l.forward(i, explain=True)

torch.testing.assert_close(out, outs)
torch.testing.assert_close(out[-1, None], torch.tensor(lstm.s)[None])

In [15]:
(out.sum() / 256).backward()

In [16]:
R_x = i.grad
torch.testing.assert_close(R_x, torch.tensor(Rx))

In [17]:
R_x.shape

torch.Size([3, 256])